In [8]:
import pandas as pd

bids_df = pd.read_csv('data1/bids.csv')
train_df = pd.read_csv('data1/train.csv')
test_df = pd.read_csv("data1/test.csv")

In [9]:
#Apply to bids_df
#most popular device
def is_device_most_popz(device):
    if(device == 'phone375'):
        return 1
    else:
        return 0
#most popular country
def is_country_most_popz(country):
    if(country == 'in'):
        return 1
    else:
        return 0

In [10]:
# bids_df['new_device'] = bids_df['device'].apply(func = is_device_most_popz)
# bids_df['new_country'] = bids_df['country'].apply(func = is_country_most_popz)

In [11]:
bids_df_unique_counts = bids_df.groupby("bidder_id").nunique()

In [12]:
# bids_df_unique_counts.drop(["bidder_id",'merchandise'],axis = 1,inplace=True)

In [13]:
inner_join_result = train_df.merge(bids_df_unique_counts, on = 'bidder_id', how = 'left')

In [14]:
# #creating probability of device being the most popular device
# inner_join_result['popz_device_proba'] = inner_join_result['new_device']/inner_join_result['device']

# #creating probability of country being the most popular country
# inner_join_result['popz_country_proba'] = inner_join_result['new_country']/inner_join_result['country']

In [15]:
bids_df_count = bids_df.groupby("bidder_id")
counts = bids_df_count['url'].count().reset_index().rename(columns = {'url':'num_bids'})

In [16]:
new_train_df = inner_join_result.merge(counts,how='left')

In [17]:
new_train_df.drop("time",inplace=True,axis=1)

In [18]:
import pandas as pd
import numpy as np
# new_train_df = new_train_df.replace([np.inf, -np.inf], np.nan)

In [19]:
new_train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
outcome,2013.0,0.051167,0.220394,0.0,0.0,0.0,0.00,1.0
bid_id,1984.0,1547.995968,14701.515067,1.0,3.0,18.0,211.75,515033.0
auction,1984.0,62.585685,147.306644,1.0,2.0,10.0,46.25,1623.0
merchandise,1984.0,1.000504,0.022451,1.0,1.0,1.0,1.00,2.0
device,1984.0,78.602319,187.731137,1.0,2.0,8.0,59.00,2618.0
country,1984.0,13.300403,23.437549,0.0,1.0,3.0,12.00,178.0
ip,1984.0,675.043347,4790.427419,1.0,2.0,12.0,116.25,111918.0
url,1984.0,346.057964,2676.988261,1.0,1.0,5.0,40.00,81376.0
num_bids,1984.0,1547.995968,14701.515067,1.0,3.0,18.0,211.75,515033.0


## TIME



In [20]:
bids_df_sorted = bids_df.sort_values(by=['bidder_id','time'],ascending = [True,True])
timediff = bids_df_sorted.groupby("bidder_id")['time'].diff()
bids= pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

bids_df_sorted['timediff']= timediff
max_time = bids_df_sorted.groupby('bidder_id')['time'].max()
bids['maxtime_num'] = max_time

min_time = bids_df_sorted.groupby('bidder_id')['time'].min()
bids['mintime_num'] = min_time

max_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
max_diff = max_diff.fillna(max_diff.mean())
bids['maxdiff_num'] = max_diff

min_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
min_diff = min_diff.fillna(min_diff.mean())

bids['mindiff_num'] = min_diff
mean_diff = bids_df_sorted.groupby('bidder_id')['timediff'].mean()

mean_diff = mean_diff.fillna(mean_diff.mean())
bids['meandiff_num'] = mean_diff

median_diff = bids_df_sorted.groupby('bidder_id')['timediff'].median()
median_diff = median_diff.fillna(median_diff.mean())

bids['mediandiff_num'] = median_diff
print(bids)



                                                                   bidder_id  \
001068c415025a009fee375a12cff4fcnht8y  001068c415025a009fee375a12cff4fcnht8y   
002d229ffb247009810828f648afc2ef593rb  002d229ffb247009810828f648afc2ef593rb   
0030a2dd87ad2733e0873062e4f83954mkj86  0030a2dd87ad2733e0873062e4f83954mkj86   
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o  003180b29c6a5f8f1d84a6b7b6f7be57tjj1o   
00486a11dff552c4bd7696265724ff81yeo9v  00486a11dff552c4bd7696265724ff81yeo9v   
...                                                                      ...   
ffbc0fdfbf19a8a9116b68714138f2902cc13  ffbc0fdfbf19a8a9116b68714138f2902cc13   
ffc4e2dd2cc08249f299cab46ecbfacfobmr3  ffc4e2dd2cc08249f299cab46ecbfacfobmr3   
ffd29eb307a4c54610dd2d3d212bf3bagmmpl  ffd29eb307a4c54610dd2d3d212bf3bagmmpl   
ffd62646d600b759a985d45918bd6f0431vmz  ffd62646d600b759a985d45918bd6f0431vmz   
fff2c070d8200e0a09150bd81452ce29ngcnv  fff2c070d8200e0a09150bd81452ce29ngcnv   

                                       

In [21]:
import pandas as pd 
import numpy as np
def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))

    return e

bids_per_auction_per_ip= bids_df_sorted[['bidder_id', 'auction', 'ip']].groupby(['bidder_id','auction', 'ip']).size()
bids_per_auction_per_ip2= bids_per_auction_per_ip.groupby(['bidder_id', 'auction']).apply(log_entropy).reset_index()
median_entropy=bids_per_auction_per_ip2.groupby('bidder_id').median()
mean_entropy=bids_per_auction_per_ip2.groupby('bidder_id').mean()
std_entropy=bids_per_auction_per_ip2.groupby('bidder_id').std()
bids['mean_entropy']= mean_entropy
bids['median_entropy']= median_entropy
bids['std_entropy']= std_entropy



In [22]:
ips_per_bidder_per_auction= bids_df_sorted[['bidder_id', 'auction', 'ip']].groupby(['bidder_id','auction']).ip.nunique()
median_ips_per_bidder_per_auction= ips_per_bidder_per_auction.groupby('bidder_id').median()
mean_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').mean()
std_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_ips_per_bidder_per_auction']= median_ips_per_bidder_per_auction
bids['mean_ips_per_bidder_per_auction']= mean_ips_per_bidder_per_auction
bids['std_ips_per_bidder_per_auction']= std_ips_per_bidder_per_auction

ip_count= bids_df_sorted.groupby(['bidder_id', 'ip']).size()
ip_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['ip_entropy']= ip_entropy

In [23]:
country_per_bidder_per_auction= bids_df_sorted[['bidder_id', 'auction', 'country']].groupby(['bidder_id','auction']).country.nunique()
median_country_per_bidder_per_auction= country_per_bidder_per_auction.groupby('bidder_id').median()
mean_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').mean()
std_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_country_per_bidder_per_auction']= median_country_per_bidder_per_auction
bids['mean_country_per_bidder_per_auction']= mean_country_per_bidder_per_auction
bids['std_country_per_bidder_per_auction']= std_country_per_bidder_per_auction
country_count= bids_df_sorted.groupby(['bidder_id', 'country']).size()
country_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['country_entropy']= country_entropy

In [24]:
device_per_bidder_per_auction= bids_df_sorted[['bidder_id', 'auction', 'device']].groupby(['bidder_id','auction']).device.nunique()
median_devices_per_bidder_per_auction= device_per_bidder_per_auction.groupby('bidder_id').median()
mean_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').mean()
std_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_devices_per_bidder_per_auction']= median_devices_per_bidder_per_auction
bids['mean_devices_per_bidder_per_auction']= mean_devices_per_bidder_per_auction
bids['std_devices_per_bidder_per_auction']= std_devices_per_bidder_per_auction
device_count= bids_df_sorted.groupby(['bidder_id', 'device']).size()
device_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['device_entropy']= device_entropy

In [25]:
try_find_time_auc = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [26]:
try_auction = bids_df.groupby(['auction'])
try_auction

In [27]:
first_bid_list = set(try_auction.first()['bidder_id'])
last_bid_list = set(try_auction.last()['bidder_id'])



In [28]:
def check_first_bid(dummy):
    count = 0
    for bidder_id in first_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
def check_last_bid(dummy):
    count = 0
    for bidder_id in last_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
    

In [29]:
# testingtestingtesting_df = test_df.copy()

In [30]:
# testingtestingtesting_df['check_first'] = testingtestingtesting_df['bidder_id'].apply(func = check_first_bid)

In [31]:
new_train_df = new_train_df.merge(bids, how = 'left', left_on = ['bidder_id'], right_on = ['bidder_id'])


In [32]:
new_train_df['country'].replace({0:1},inplace=True)

In [33]:
new_train_df['bids_per_auction'] = new_train_df['num_bids']/new_train_df['auction']
new_train_df['bids_per_url'] = new_train_df['num_bids']/new_train_df['url']
new_train_df['bids_per_ip'] = new_train_df['num_bids']/new_train_df['ip']
new_train_df['bids_per_country'] = new_train_df['num_bids']/new_train_df['country']

In [34]:
#end result new_train
new_train_df.columns


Index(['bidder_id', 'payment_account', 'address', 'outcome', 'bid_id',
       'auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country'],
      dtype='object')

## DIVERSITY (COUNTRY)


In [35]:
diversity_bids = bids_df.sort_values(['bidder_id','country'])

In [36]:
num_countries_per_auction = diversity_bids.groupby(['bidder_id','auction'])['country'].nunique()
num_countries_per_auction = pd.DataFrame(num_countries_per_auction)

In [37]:
mean_country_per_auction = num_countries_per_auction.groupby("bidder_id").mean()
mean_country_per_auction = mean_country_per_auction.reset_index().rename(columns = {'country':"mean_country_per_auction"})

max_country_per_auction = num_countries_per_auction.groupby("bidder_id").max()
max_country_per_auction = max_country_per_auction.reset_index().rename(columns = {'country':"max_country_per_auction"})

min_country_per_auction = num_countries_per_auction.groupby("bidder_id").min()
min_country_per_auction = min_country_per_auction.reset_index().rename(columns = {'country':"min_country_per_auction"})

std_country_per_auction = num_countries_per_auction.groupby("bidder_id").std()
std_country_per_auction = std_country_per_auction.reset_index().rename(columns = {'country':"std_country_per_auction"})


In [38]:
# add_train_df = new_train_df.merge(mean_country_per_auction,on='bidder_id',how='left')
# add_train_df = add_train_df.merge(max_country_per_auction,on='bidder_id',how='left')
# add_train_df = add_train_df.merge(min_country_per_auction,on='bidder_id',how='left')
# add_train_df = add_train_df.merge(std_country_per_auction,on='bidder_id',how='left')

In [39]:
# add_train_df.fillna(0,inplace=True)


In [40]:
country_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [41]:
country_bids = country_bids.merge(mean_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(max_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(min_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(std_country_per_auction,on='bidder_id',how='left')

In [42]:
# country_bids

## DIVERSITY (DEVICE)

In [43]:
device_bids = bids_df.sort_values(['bidder_id','device'])
num_devices_per_auction = device_bids.groupby(['bidder_id','auction'])['device'].nunique()
num_devices_per_auction = pd.DataFrame(num_devices_per_auction)

In [44]:
mean_devices_per_auction = num_devices_per_auction.groupby("bidder_id").mean()
mean_devices_per_auction = mean_devices_per_auction.reset_index().rename(columns = {'device':"mean_devices_per_auction"})

max_devices_per_auction = num_devices_per_auction.groupby("bidder_id").max()
max_devices_per_auction = max_devices_per_auction.reset_index().rename(columns = {'device':"max_devices_per_auction"})

min_devices_per_auction = num_devices_per_auction.groupby("bidder_id").min()
min_devices_per_auction = min_devices_per_auction.reset_index().rename(columns = {'device':"min_devices_per_auction"})

std_devices_per_auction = num_devices_per_auction.groupby("bidder_id").std()
std_devices_per_auction = std_devices_per_auction.reset_index().rename(columns = {'device':"std_devices_per_auction"})


In [45]:
device_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [46]:
device_bids = device_bids.merge(mean_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(max_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(min_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(std_devices_per_auction,on='bidder_id',how='left')

In [47]:
device_bids.fillna(0,inplace=True)

## DIVERSITY (IP)

In [48]:
ip_bids = bids_df.sort_values(['bidder_id','ip'])
num_ip_per_auction = ip_bids.groupby(['bidder_id','auction'])['ip'].nunique()
num_ip_per_auction = pd.DataFrame(num_ip_per_auction)

In [49]:
mean_ip_per_auction = num_ip_per_auction.groupby("bidder_id").mean()
mean_ip_per_auction = mean_ip_per_auction.reset_index().rename(columns = {'ip':"mean_ip_per_auction"})

max_ip_per_auction = num_ip_per_auction.groupby("bidder_id").max()
max_ip_per_auction = max_ip_per_auction.reset_index().rename(columns = {'ip':"max_ip_per_auction"})

min_ip_per_auction = num_ip_per_auction.groupby("bidder_id").min()
min_ip_per_auction = min_ip_per_auction.reset_index().rename(columns = {'ip':"min_ip_per_auction"})

std_ip_per_auction = num_ip_per_auction.groupby("bidder_id").std()
std_ip_per_auction = std_ip_per_auction.reset_index().rename(columns = {'ip':"std_ip_per_auction"})


In [50]:
ip_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [51]:
ip_bids = ip_bids.merge(mean_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(max_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(min_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(std_ip_per_auction,on='bidder_id',how='left')

In [52]:
ip_bids.fillna(0,inplace=True)

## DOING THE TRAIN DATASET


In [142]:
training_model_df = new_train_df.copy()


In [54]:
# training_model_df = training_model_df.merge(bids,on='bidder_id',how='left')

In [143]:
training_model_df = training_model_df.merge(country_bids,on='bidder_id',how='left')

In [144]:
training_model_df = training_model_df.merge(device_bids,on='bidder_id',how='left')

In [145]:
training_model_df = training_model_df.merge(ip_bids,on='bidder_id',how='left')

In [146]:
training_model_df['check_first'] = training_model_df['bidder_id'].apply(func = check_first_bid)
training_model_df['check_last'] = training_model_df['bidder_id'].apply(func = check_last_bid)

In [147]:
training_model_df.fillna(0,inplace=True)

In [148]:
bots_training_model_df = training_model_df[training_model_df['outcome'] == 1]
humans_training_model_df = training_model_df[training_model_df['outcome'] == 0]
bots_training_model_df = bots_training_model_df[bots_training_model_df['num_bids'] > 1]

In [149]:
training_model_df = pd.concat([bots_training_model_df,humans_training_model_df])

In [150]:
training_model_df = training_model_df.reset_index()

In [151]:
training_model_df.drop(columns = ['index'],inplace = True)

In [64]:
# from sklearn.preprocessing import StandardScaler
# scaling_features = ['auction', 'device', 'country', 'ip', 'url', 'popz_device_proba',
#        'popz_country_proba', 'num_bids', 'maxtime_num', 'mintime_num',
#        'maxdiff_num', 'mindiff_num', 'meandiff_num', 'mediandiff_num',
#        'mean_entropy', 'median_entropy', 'std_entropy',
#        'median_ips_per_bidder_per_auction', 'mean_ips_per_bidder_per_auction',
#        'std_ips_per_bidder_per_auction', 'ip_entropy',
#        'median_country_per_bidder_per_auction',
#        'mean_country_per_bidder_per_auction',
#        'std_country_per_bidder_per_auction', 'country_entropy',
#        'median_devices_per_bidder_per_auction',
#        'mean_devices_per_bidder_per_auction',
#        'std_devices_per_bidder_per_auction', 'device_entropy', 'bids_per_ip',
#        'bids_per_country', 'mean_devices_per_auction',
#        'max_devices_per_auction', 'min_devices_per_auction',
#        'std_devices_per_auction', 'check_first', 'check_last']

# scaler = StandardScaler()
# data = training_model_df[scaling_features]
# scaler.fit(data)
# training_model_df[scaling_features] = scaler.transform(data)



In [152]:
training_model_df.describe().T

,count,mean,std,min,25%,50%,75%,max
outcome,2008.0,4.880478e-02,2.155134e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
bid_id,2008.0,1.529492e+03,1.461432e+04,0.0,3.000000e+00,1.700000e+01,2.010000e+02,5.150330e+05
auction,2008.0,6.183516e+01,1.465822e+02,0.0,2.000000e+00,1.000000e+01,4.600000e+01,1.623000e+03
merchandise,2008.0,9.860558e-01,1.214625e-01,0.0,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
device,2008.0,7.766036e+01,1.868018e+02,0.0,2.000000e+00,8.000000e+00,5.725000e+01,2.618000e+03
country,2008.0,1.313944e+01,2.334289e+01,0.0,1.000000e+00,3.000000e+00,1.200000e+01,1.780000e+02
ip,2008.0,6.669726e+02,4.762265e+03,0.0,2.000000e+00,1.200000e+01,1.122500e+02,1.119180e+05
url,2008.0,3.419193e+02,2.661200e+03,0.0,1.000000e+00,5.000000e+00,3.800000e+01,8.137600e+04
num_bids,2008.0,1.529492e+03,1.461432e+04,0.0,3.000000e+00,1.700000e+01,2.010000e+02,5.150330e+05
maxtime_num,2008.0,9.587146e+15,1.161618e+15,0.0,9.706561e+15,9.709211e+15,9.771565e+15,9.772885e+15


In [153]:
len(training_model_df.columns)

51

In [154]:
corr = training_model_df.corr()

corr[:1].T

,outcome
outcome,1.000000
bid_id,0.041530
auction,0.139967
merchandise,0.026011
device,0.114313
country,0.140476
ip,0.087664
url,0.019616
num_bids,0.041530
maxtime_num,0.028434


## TESTING TRAIN DATASET


In [155]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import statistics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [156]:
y = training_model_df['outcome']
X = training_model_df.drop(columns= ['bid_id',
                                     'outcome', 
                                     'bidder_id',
                                     'payment_account',
                                     'address', 
                                     
                                    
                                     ])

In [157]:
len(X.columns)

46

In [158]:
training_model_df.columns

Index(['bidder_id', 'payment_account', 'address', 'outcome', 'bid_id',
       'auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_dev

In [159]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
auction,2008.0,6.183516e+01,1.465822e+02,0.0,2.000000e+00,1.000000e+01,4.600000e+01,1.623000e+03
merchandise,2008.0,9.860558e-01,1.214625e-01,0.0,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
device,2008.0,7.766036e+01,1.868018e+02,0.0,2.000000e+00,8.000000e+00,5.725000e+01,2.618000e+03
country,2008.0,1.313944e+01,2.334289e+01,0.0,1.000000e+00,3.000000e+00,1.200000e+01,1.780000e+02
ip,2008.0,6.669726e+02,4.762265e+03,0.0,2.000000e+00,1.200000e+01,1.122500e+02,1.119180e+05
url,2008.0,3.419193e+02,2.661200e+03,0.0,1.000000e+00,5.000000e+00,3.800000e+01,8.137600e+04
num_bids,2008.0,1.529492e+03,1.461432e+04,0.0,3.000000e+00,1.700000e+01,2.010000e+02,5.150330e+05
maxtime_num,2008.0,9.587146e+15,1.161618e+15,0.0,9.706561e+15,9.709211e+15,9.771565e+15,9.772885e+15
mintime_num,2008.0,9.556047e+15,1.158649e+15,0.0,9.632360e+15,9.696580e+15,9.759414e+15,9.772871e+15
maxdiff_num,2008.0,2.407148e+13,2.347518e+13,0.0,2.007276e+12,2.428943e+13,5.017759e+13,7.610295e+13


In [160]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.3, random_state= 20)

In [163]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1808, 46)
(200, 46)
(1808,)
(200,)


In [162]:
model1 = RandomForestClassifier(n_estimators=160, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)

model2 = xgb.XGBClassifier( objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)

model3 = GradientBoostingClassifier(n_estimators=200,
                                       random_state=20,
                                       max_depth=5,
                                       learning_rate=0.03,
                                       max_features=5,)
model4 = CatBoostClassifier(random_state=20)

k_fold_accuracy = []
kf = KFold(n_splits=10,shuffle = True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model1.fit(X_train,y_train)
    model2.fit(X_train,y_train)
    model3.fit(X_train,y_train)
    model4.fit(X_train,y_train)
#     model5.fit(X_train,y_train)

    y_pred1 = model1.predict_proba(X_test)
    y_pred2 = model2.predict_proba(X_test)
    y_pred3 = model3.predict_proba(X_test)
    y_pred4 = model4.predict_proba(X_test)
#     y_pred5 = model5.predict_proba(X_test)
#     final_y_pred =( y_pred1+y_pred2+y_pred3+y_pred4+y_pred5)/5
    
    accu1 = model1.predict(X_test)
    accu2 = model2.predict(X_test)
    accu3 = model3.predict(X_test)
    accu4 = model4.predict(X_test)
#     accu5 = model5.predict(X_test)
#     final_y_accu = (accu1+accu2+accu3+accu4+accu5)/5

/Users/brennanszeto/Environments/anaconda/miniconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:59:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Learning rate set to 0.013263
0:	learn: 0.6609339	total: 7.82ms	remaining: 7.81s
1:	learn: 0.6399229	total: 12.4ms	remaining: 6.21s
2:	learn: 0.6169875	total: 16.7ms	remaining: 5.54s
3:	learn: 0.5919958	total: 19.7ms	remaining: 4.91s
4:	learn: 0.5674415	total: 23.3ms	remaining: 4.64s
5:	learn: 0.5509238	total: 28.7ms	remaining: 4.76s
6:	learn: 0.5266532	total: 33.9ms	remaining: 4.81s
7:	learn: 0.5081116	total: 38ms	remaining: 4.71s
8:	learn: 0.4872788	total: 41.6ms	remaining: 4.58s
9:	learn: 0.4698152	total: 46.3ms	remaining: 4.59s
10:	learn: 0.4520543	total: 51.1ms	remaining: 4.59s
11:	learn: 0.4372586	total: 55.6ms	remaining: 4.57s
12

202:	learn: 0.0659329	total: 768ms	remaining: 3.02s
203:	learn: 0.0656455	total: 773ms	remaining: 3.02s
204:	learn: 0.0655444	total: 777ms	remaining: 3.01s
205:	learn: 0.0653751	total: 782ms	remaining: 3.01s
206:	learn: 0.0652011	total: 786ms	remaining: 3.01s
207:	learn: 0.0650355	total: 790ms	remaining: 3.01s
208:	learn: 0.0648898	total: 795ms	remaining: 3.01s
209:	learn: 0.0647145	total: 798ms	remaining: 3s
210:	learn: 0.0644727	total: 803ms	remaining: 3s
211:	learn: 0.0641413	total: 807ms	remaining: 3s
212:	learn: 0.0638324	total: 810ms	remaining: 2.99s
213:	learn: 0.0637156	total: 814ms	remaining: 2.99s
214:	learn: 0.0636091	total: 817ms	remaining: 2.98s
215:	learn: 0.0634710	total: 820ms	remaining: 2.97s
216:	learn: 0.0632633	total: 823ms	remaining: 2.97s
217:	learn: 0.0631071	total: 826ms	remaining: 2.96s
218:	learn: 0.0630112	total: 829ms	remaining: 2.96s
219:	learn: 0.0627132	total: 832ms	remaining: 2.95s
220:	learn: 0.0626072	total: 835ms	remaining: 2.94s
221:	learn: 0.0625282

383:	learn: 0.0419398	total: 1.54s	remaining: 2.47s
384:	learn: 0.0417840	total: 1.55s	remaining: 2.47s
385:	learn: 0.0416994	total: 1.55s	remaining: 2.47s
386:	learn: 0.0416020	total: 1.56s	remaining: 2.47s
387:	learn: 0.0415613	total: 1.56s	remaining: 2.46s
388:	learn: 0.0414882	total: 1.57s	remaining: 2.46s
389:	learn: 0.0414268	total: 1.57s	remaining: 2.46s
390:	learn: 0.0413712	total: 1.57s	remaining: 2.45s
391:	learn: 0.0413363	total: 1.58s	remaining: 2.45s
392:	learn: 0.0412090	total: 1.59s	remaining: 2.45s
393:	learn: 0.0411471	total: 1.59s	remaining: 2.45s
394:	learn: 0.0410758	total: 1.6s	remaining: 2.44s
395:	learn: 0.0410305	total: 1.6s	remaining: 2.44s
396:	learn: 0.0409924	total: 1.6s	remaining: 2.44s
397:	learn: 0.0409441	total: 1.61s	remaining: 2.44s
398:	learn: 0.0408350	total: 1.61s	remaining: 2.43s
399:	learn: 0.0407423	total: 1.62s	remaining: 2.43s
400:	learn: 0.0406728	total: 1.62s	remaining: 2.42s
401:	learn: 0.0405989	total: 1.63s	remaining: 2.42s
402:	learn: 0.0

554:	learn: 0.0306014	total: 2.52s	remaining: 2.02s
555:	learn: 0.0305150	total: 2.53s	remaining: 2.02s
556:	learn: 0.0304649	total: 2.53s	remaining: 2.01s
557:	learn: 0.0303865	total: 2.53s	remaining: 2.01s
558:	learn: 0.0303626	total: 2.54s	remaining: 2s
559:	learn: 0.0303373	total: 2.54s	remaining: 2s
560:	learn: 0.0302552	total: 2.55s	remaining: 1.99s
561:	learn: 0.0301710	total: 2.55s	remaining: 1.99s
562:	learn: 0.0301277	total: 2.56s	remaining: 1.99s
563:	learn: 0.0300932	total: 2.56s	remaining: 1.98s
564:	learn: 0.0300764	total: 2.57s	remaining: 1.98s
565:	learn: 0.0300103	total: 2.57s	remaining: 1.97s
566:	learn: 0.0299620	total: 2.57s	remaining: 1.97s
567:	learn: 0.0299021	total: 2.58s	remaining: 1.96s
568:	learn: 0.0298476	total: 2.58s	remaining: 1.96s
569:	learn: 0.0297937	total: 2.59s	remaining: 1.95s
570:	learn: 0.0297638	total: 2.59s	remaining: 1.95s
571:	learn: 0.0297289	total: 2.6s	remaining: 1.94s
572:	learn: 0.0296922	total: 2.6s	remaining: 1.94s
573:	learn: 0.029670

...

161:	learn: 0.0837086	total: 563ms	remaining: 2.91s
162:	learn: 0.0835048	total: 567ms	remaining: 2.91s
163:	learn: 0.0831889	total: 570ms	remaining: 2.91s
164:	learn: 0.0828818	total: 574ms	remaining: 2.9s
165:	learn: 0.0826936	total: 577ms	remaining: 2.9s
166:	learn: 0.0823858	total: 580ms	remaining: 2.9s
167:	learn: 0.0819886	total: 583ms	remaining: 2.89s
168:	learn: 0.0817017	total: 587ms	remaining: 2.88s
169:	learn: 0.0814157	total: 590ms	remaining: 2.88s
170:	learn: 0.0811969	total: 593ms	remaining: 2.87s
171:	learn: 0.0810942	total: 596ms	remaining: 2.87s
172:	learn: 0.0808571	total: 599ms	remaining: 2.87s
173:	learn: 0.0807150	total: 603ms	remaining: 2.86s
174:	learn: 0.0804207	total: 606ms	remaining: 2.86s
175:	learn: 0.0803426	total: 609ms	remaining: 2.85s
176:	learn: 0.0800147	total: 613ms	remaining: 2.85s
177:	learn: 0.0797339	total: 616ms	remaining: 2.84s
178:	learn: 0.0794582	total: 619ms	remaining: 2.84s
179:	learn: 0.0791166	total: 622ms	remaining: 2.83s
180:	learn: 0.0

334:	learn: 0.0526205	total: 1.35s	remaining: 2.69s
335:	learn: 0.0525187	total: 1.36s	remaining: 2.68s
336:	learn: 0.0524774	total: 1.36s	remaining: 2.68s
337:	learn: 0.0523633	total: 1.36s	remaining: 2.67s
338:	learn: 0.0522710	total: 1.37s	remaining: 2.67s
339:	learn: 0.0521522	total: 1.37s	remaining: 2.66s
340:	learn: 0.0520657	total: 1.37s	remaining: 2.66s
341:	learn: 0.0519033	total: 1.38s	remaining: 2.65s
342:	learn: 0.0518774	total: 1.38s	remaining: 2.65s
343:	learn: 0.0517373	total: 1.38s	remaining: 2.64s
344:	learn: 0.0516730	total: 1.39s	remaining: 2.63s
345:	learn: 0.0515953	total: 1.39s	remaining: 2.63s
346:	learn: 0.0515422	total: 1.39s	remaining: 2.63s
347:	learn: 0.0514138	total: 1.4s	remaining: 2.62s
348:	learn: 0.0512694	total: 1.4s	remaining: 2.61s
349:	learn: 0.0511655	total: 1.41s	remaining: 2.61s
350:	learn: 0.0510453	total: 1.41s	remaining: 2.6s
351:	learn: 0.0509830	total: 1.41s	remaining: 2.6s
352:	learn: 0.0509357	total: 1.42s	remaining: 2.59s
353:	learn: 0.05

529:	learn: 0.0360189	total: 2.12s	remaining: 1.88s
530:	learn: 0.0359548	total: 2.13s	remaining: 1.88s
531:	learn: 0.0359236	total: 2.13s	remaining: 1.87s
532:	learn: 0.0359102	total: 2.13s	remaining: 1.87s
533:	learn: 0.0358782	total: 2.14s	remaining: 1.86s
534:	learn: 0.0358116	total: 2.14s	remaining: 1.86s
535:	learn: 0.0357732	total: 2.14s	remaining: 1.85s
536:	learn: 0.0356814	total: 2.15s	remaining: 1.85s
537:	learn: 0.0356429	total: 2.15s	remaining: 1.85s
538:	learn: 0.0355658	total: 2.15s	remaining: 1.84s
539:	learn: 0.0354912	total: 2.16s	remaining: 1.84s
540:	learn: 0.0354338	total: 2.16s	remaining: 1.83s
541:	learn: 0.0353872	total: 2.16s	remaining: 1.83s
542:	learn: 0.0352914	total: 2.17s	remaining: 1.82s
543:	learn: 0.0352374	total: 2.17s	remaining: 1.82s
544:	learn: 0.0352026	total: 2.17s	remaining: 1.81s
545:	learn: 0.0351186	total: 2.18s	remaining: 1.81s
546:	learn: 0.0350683	total: 2.18s	remaining: 1.8s
547:	learn: 0.0350362	total: 2.18s	remaining: 1.8s
548:	learn: 0.

701:	learn: 0.0269992	total: 2.9s	remaining: 1.23s
702:	learn: 0.0269600	total: 2.9s	remaining: 1.23s
703:	learn: 0.0269132	total: 2.91s	remaining: 1.22s
704:	learn: 0.0268665	total: 2.91s	remaining: 1.22s
705:	learn: 0.0268327	total: 2.92s	remaining: 1.21s
706:	learn: 0.0268131	total: 2.92s	remaining: 1.21s
707:	learn: 0.0267775	total: 2.92s	remaining: 1.21s
708:	learn: 0.0267492	total: 2.93s	remaining: 1.2s
709:	learn: 0.0267137	total: 2.93s	remaining: 1.2s
710:	learn: 0.0266297	total: 2.93s	remaining: 1.19s
711:	learn: 0.0265962	total: 2.94s	remaining: 1.19s
712:	learn: 0.0265712	total: 2.94s	remaining: 1.18s
713:	learn: 0.0265329	total: 2.94s	remaining: 1.18s
714:	learn: 0.0264696	total: 2.95s	remaining: 1.18s
715:	learn: 0.0264543	total: 2.95s	remaining: 1.17s
716:	learn: 0.0264429	total: 2.95s	remaining: 1.17s
717:	learn: 0.0264202	total: 2.96s	remaining: 1.16s
718:	learn: 0.0263707	total: 2.96s	remaining: 1.16s
719:	learn: 0.0263261	total: 2.96s	remaining: 1.15s
720:	learn: 0.02

885:	learn: 0.0209725	total: 3.66s	remaining: 471ms
886:	learn: 0.0209450	total: 3.69s	remaining: 471ms
887:	learn: 0.0209132	total: 3.7s	remaining: 467ms
888:	learn: 0.0208515	total: 3.7s	remaining: 463ms
889:	learn: 0.0208136	total: 3.71s	remaining: 458ms
890:	learn: 0.0207904	total: 3.71s	remaining: 454ms
891:	learn: 0.0207305	total: 3.72s	remaining: 450ms
892:	learn: 0.0207028	total: 3.72s	remaining: 446ms
893:	learn: 0.0206883	total: 3.73s	remaining: 442ms
894:	learn: 0.0206634	total: 3.73s	remaining: 438ms
895:	learn: 0.0206348	total: 3.73s	remaining: 434ms
896:	learn: 0.0206207	total: 3.74s	remaining: 429ms
897:	learn: 0.0205789	total: 3.74s	remaining: 425ms
898:	learn: 0.0205721	total: 3.75s	remaining: 421ms
899:	learn: 0.0205415	total: 3.75s	remaining: 417ms
900:	learn: 0.0205348	total: 3.75s	remaining: 412ms
901:	learn: 0.0205055	total: 3.76s	remaining: 408ms
902:	learn: 0.0204675	total: 3.76s	remaining: 404ms
903:	learn: 0.0204196	total: 3.76s	remaining: 400ms
904:	learn: 0.

In [74]:
from sklearn.metrics import accuracy_score


print('\nAccuracy_score on RandomForest : ', accuracy_score(y_test, accu1))
print('\nAccuracy_score on XGB : ', accuracy_score(y_test, accu2))
print('\nAccuracy_score on GradientBoost : ', accuracy_score(y_test, accu3))
print('\nAccuracy_score on CatBoost : ', accuracy_score(y_test, accu4))

averaging_accuracy = (accuracy_score(y_test, accu1) + accuracy_score(y_test, accu2) +accuracy_score(y_test, accu3)+accuracy_score(y_test, accu4))/4
print('\nAverage Accuracy : ', averaging_accuracy)

# print('\Final Accuracy : ', accuracy_score(y_test, final_y_accu))


Accuracy_score on RandomForest :  0.965

Accuracy_score on XGB :  0.965

Accuracy_score on GradientBoost :  0.965

Accuracy_score on CatBoost :  0.96

Average Accuracy :  0.96375


In [75]:
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
# roc_curve(y_test,y_predict_proba)
print("RandomForest : " , roc_auc_score(y_test,y_pred1[:,1]))

print("XGB : " , roc_auc_score(y_test,y_pred2[:,1]))
print("Gradient Boost : " , roc_auc_score(y_test,y_pred3[:,1]))
print("Cat Boost : ", roc_auc_score(y_test,y_pred4[:,1]))
# print("Cat Boost : ", roc_auc_score(y_test,y_pred5[:,1]))
score1 = roc_auc_score(y_test,y_pred1[:,1])
score2 = roc_auc_score(y_test,y_pred2[:,1])
score3 = roc_auc_score(y_test,y_pred3[:,1])
score4 = roc_auc_score(y_test,y_pred4[:,1])
# score5 = roc_auc_score(y_test,y_pred5[:,1])
print("\nAverage AUC :" , (score1+score2+score3+score4)/4)
# print(roc_auc_score(y_test,final_y_pred[:,1]))

RandomForest :  0.9240837696335079
XGB :  0.8743455497382198
Gradient Boost :  0.9185573007562536
Cat Boost :  0.9243746364165214

Average AUC : 0.9103403141361257


In [76]:
X.columns

Index(['auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_a

In [77]:
X.shape

(2008, 46)

## MAKING OF FINAL TEST_DF


In [175]:
final_test_df = test_df.merge(bids_df_unique_counts,how= 'left',on = 'bidder_id')

In [79]:
# final_test_df['popz_device_proba'] = final_test_df['new_device']/final_test_df['device']

# #creating probability of country being the most popular country
# final_test_df['popz_country_proba'] = final_test_df['new_country']/final_test_df['country']

In [80]:
# final_test_df = final_test_df.replace([np.inf, -np.inf], np.nan)

In [176]:
final_test_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bid_id,4630.0,990.304536,6227.856448,1.0,3.0,18.0,179.00,168995.0
auction,4630.0,55.760475,124.486828,1.0,2.0,10.0,47.00,1726.0
merchandise,4630.0,1.000000,0.000000,1.0,1.0,1.0,1.00,1.0
device,4630.0,71.224406,164.947305,1.0,2.0,8.0,56.75,2069.0
time,4630.0,918.739957,5334.479886,1.0,3.0,18.0,177.75,135480.0
country,4630.0,12.477322,22.157906,0.0,1.0,3.0,12.00,171.0
ip,4630.0,487.984233,2525.546673,1.0,2.0,12.0,107.00,58150.0
url,4630.0,267.042765,2000.302904,1.0,1.0,5.0,35.00,73417.0


In [177]:
final_test_df = final_test_df.merge(counts, how = 'left', on='bidder_id')

In [178]:
final_test_df = final_test_df.merge(bids,how='left', on ='bidder_id')

In [179]:
final_test_df.fillna(0,inplace=True)

In [85]:
# training_model_df.columns

In [182]:
final_test_df['country'].replace({0:1},inplace=True)

In [183]:
final_test_df['bids_per_auction'] = final_test_df['num_bids']/final_test_df['auction']
final_test_df['bids_per_url'] = final_test_df['num_bids']/final_test_df['url']
final_test_df['bids_per_ip'] = final_test_df['num_bids']/final_test_df['ip']
final_test_df['bids_per_country'] = final_test_df['num_bids']/final_test_df['country']

In [184]:
final_test_df = final_test_df.merge(country_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(device_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(ip_bids,on='bidder_id',how='left')

In [185]:
final_test_df['check_first'] = final_test_df['bidder_id'].apply(func = check_first_bid)
final_test_df['check_last'] = final_test_df['bidder_id'].apply(func = check_last_bid)

In [186]:
final_test_df.shape

(4700, 51)

## TESTING FINAL TEST_DF

In [187]:
X_train = X
y_train = y

In [206]:
X_train

,auction,merchandise,device,country,ip,url,num_bids,maxtime_num,mintime_num,maxdiff_num,...,mean_devices_per_auction,max_devices_per_auction,min_devices_per_auction,std_devices_per_auction,mean_ip_per_auction,max_ip_per_auction,min_ip_per_auction,std_ip_per_auction,check_first,check_last
0,18.0,1.0,51.0,7.0,13.0,1.0,181.0,9.645532e+15,9.632357e+15,8.512737e+12,...,7.222222,13.0,1.0,4.180565,1.777778,4.0,1.0,0.942809,0,0
1,269.0,1.0,408.0,65.0,3455.0,1916.0,6540.0,9.709222e+15,9.631919e+15,5.002937e+13,...,3.869888,369.0,1.0,22.512554,14.026022,2975.0,1.0,181.247695,1,0
2,127.0,1.0,24.0,14.0,358.0,5.0,2048.0,9.709222e+15,9.631920e+15,5.010521e+13,...,2.732283,15.0,1.0,2.595628,8.535433,96.0,1.0,17.560234,1,1
3,26.0,1.0,1.0,2.0,8.0,7.0,281.0,9.771180e+15,9.767714e+15,3.268789e+12,...,1.000000,1.0,1.0,0.000000,2.384615,4.0,1.0,1.061204,0,1
4,107.0,1.0,4.0,29.0,836.0,4.0,1103.0,9.772867e+15,9.759244e+15,1.931579e+11,...,1.504673,4.0,1.0,0.828431,8.962617,111.0,1.0,17.386248,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,25.0,1.0,4.0,4.0,5.0,2.0,36.0,9.707030e+15,9.633559e+15,5.605500e+13,...,1.000000,1.0,1.0,0.000000,1.000000,1.0,1.0,0.000000,1,0
2004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.705152e+15,9.705152e+15,2.428943e+13,...,1.000000,1.0,1.0,0.000000,1.000000,1.0,1.0,0.000000,0,0
2005,1.0,1.0,2.0,1.0,2.0,1.0,2.0,9.772601e+15,9.762802e+15,9.798474e+12,...,2.000000,2.0,2.0,0.000000,2.000000,2.0,2.0,0.000000,0,0
2006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.636754e+15,9.636754e+15,2.428943e+13,...,1.000000,1.0,1.0,0.000000,1.000000,1.0,1.0,0.000000,0,0


In [115]:
#next test feature
testing_feature.columns()

In [208]:
# checking which feature to use
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
data = final_test_df
X=X_train
y=y_train
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores)
# print(featureScores.nlargest(10,'Score'))  #print 10 best features

                                    Specs         Score
0                                 auction  1.366227e+04
1                             merchandise  2.031625e-02
2                                  device  1.178418e+04
3                                 country  1.642422e+03
4                                      ip  5.244571e+05
5                                     url  1.599508e+04
6                                num_bids  4.833756e+05
7                             maxtime_num  2.283833e+14
8                             mintime_num  2.331751e+14
9                             maxdiff_num  2.768748e+14
10                            mindiff_num  2.768748e+14
11                           meandiff_num  3.184604e+14
12                         mediandiff_num  1.920825e+14
13                           mean_entropy  1.102484e+04
14                         median_entropy  1.134793e+00
15                            std_entropy  1.030441e+05
16      median_ips_per_bidder_per_auction  7.216

In [190]:
#46 features used [ MAIN ]
testing_feature = ['auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'check_first', 'check_last']

In [209]:
X_train.columns

Index(['auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_a

In [198]:
#46 features used [ MAIN ]
testing_feature = ['auction', 'merchandise', 'device', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'mean_entropy', 'median_entropy',
       'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'bids_per_auction', 'bids_per_url', 'bids_per_ip', 'bids_per_country',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'check_first', 'check_last']

In [199]:
X_test = final_test_df[testing_feature]

In [200]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
auction,4700.0,5.493000e+01,1.237406e+02,0.0,2.000000e+00,9.000000e+00,4.600000e+01,1.726000e+03
merchandise,4700.0,9.851064e-01,1.211401e-01,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
device,4700.0,7.016362e+01,1.639413e+02,0.0,2.000000e+00,8.000000e+00,5.500000e+01,2.069000e+03
country,4700.0,1.230723e+01,2.203570e+01,1.0,1.000000e+00,3.000000e+00,1.200000e+01,1.710000e+02
ip,4700.0,4.807164e+02,2.507362e+03,0.0,2.000000e+00,1.100000e+01,1.020000e+02,5.815000e+04
url,4700.0,2.630655e+02,1.985611e+03,0.0,1.000000e+00,5.000000e+00,3.300000e+01,7.341700e+04
num_bids,4700.0,9.755553e+02,6.182459e+03,0.0,2.000000e+00,1.700000e+01,1.712500e+02,1.689950e+05
maxtime_num,4700.0,9.584196e+15,1.179314e+15,0.0,9.706602e+15,9.709211e+15,9.771765e+15,9.772885e+15
mintime_num,4700.0,9.553830e+15,1.176385e+15,0.0,9.632510e+15,9.698435e+15,9.759488e+15,9.772874e+15
maxdiff_num,4700.0,2.387091e+13,2.335828e+13,0.0,1.871737e+12,2.428943e+13,5.016996e+13,7.600195e+13


In [193]:
X_test.isnull().sum()

auction                                    0
device                                     0
country                                    0
ip                                         0
url                                        0
num_bids                                   0
maxtime_num                                0
mintime_num                                0
maxdiff_num                                0
mindiff_num                                0
meandiff_num                               0
mediandiff_num                             0
mean_entropy                               0
median_entropy                             0
std_entropy                                0
median_ips_per_bidder_per_auction          0
mean_ips_per_bidder_per_auction            0
std_ips_per_bidder_per_auction             0
ip_entropy                                 0
median_country_per_bidder_per_auction      0
mean_country_per_bidder_per_auction        0
std_country_per_bidder_per_auction         0
country_en

In [201]:
print(X_test.shape)
X_test.fillna(0,inplace=True)

(4700, 46)


/Users/brennanszeto/Environments/anaconda/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [202]:
fmodel1 = RandomForestClassifier(n_estimators=160, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)
fmodel2 = xgb.XGBClassifier(objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)

fmodel4 = CatBoostClassifier(random_seed=20)


fmodel1.fit(X_train,y_train)
fmodel2.fit(X_train,y_train)
fmodel4.fit(X_train,y_train)

final_answer1= fmodel1.predict_proba(X_test)[:,1]

final_answer2= fmodel2.predict_proba(X_test)[:,1]
final_answer4= fmodel4.predict_proba(X_test)[:,1]


finalpredd= (final_answer1+final_answer2+final_answer4)/3
print(len(finalpredd))

/Users/brennanszeto/Environments/anaconda/miniconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:09:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Learning rate set to 0.013874
0:	learn: 0.6615696	total: 10.8ms	remaining: 10.8s
1:	learn: 0.6392648	total: 17.8ms	remaining: 8.89s
2:	learn: 0.6152695	total: 27.8ms	remaining: 9.25s
3:	learn: 0.5892241	total: 34.8ms	remaining: 8.68s
4:	learn: 0.5648755	total: 45.5ms	remaining: 9.05s
5:	learn: 0.5435224	total: 54.5ms	remaining: 9.03s
6:	learn: 0.5190852	total: 63.6ms	remaining: 9.02s
7:	learn: 0.5003855	total: 71.8ms	remaining: 8.9s
8:	learn: 0.4805589	total: 84.4ms	remaining: 9.29s
9:	learn: 0.4616810	total: 101ms	remaining: 9.98s
10:	learn: 0.4441825	total: 117ms	remaining: 10.6s
11:	learn: 0.4293725	total: 128ms	remaining: 10.6s
12:	

169:	learn: 0.0809577	total: 962ms	remaining: 4.7s
170:	learn: 0.0807353	total: 968ms	remaining: 4.69s
171:	learn: 0.0805362	total: 974ms	remaining: 4.69s
172:	learn: 0.0802888	total: 980ms	remaining: 4.69s
173:	learn: 0.0801645	total: 986ms	remaining: 4.68s
174:	learn: 0.0798715	total: 991ms	remaining: 4.67s
175:	learn: 0.0796040	total: 997ms	remaining: 4.67s
176:	learn: 0.0791637	total: 1s	remaining: 4.67s
177:	learn: 0.0788762	total: 1.01s	remaining: 4.65s
178:	learn: 0.0787432	total: 1.01s	remaining: 4.64s
179:	learn: 0.0783424	total: 1.01s	remaining: 4.63s
180:	learn: 0.0782632	total: 1.02s	remaining: 4.61s
181:	learn: 0.0782196	total: 1.02s	remaining: 4.6s
182:	learn: 0.0780098	total: 1.03s	remaining: 4.58s
183:	learn: 0.0777751	total: 1.03s	remaining: 4.57s
184:	learn: 0.0775809	total: 1.03s	remaining: 4.55s
185:	learn: 0.0772759	total: 1.04s	remaining: 4.54s
186:	learn: 0.0770905	total: 1.04s	remaining: 4.53s
187:	learn: 0.0769570	total: 1.04s	remaining: 4.51s
188:	learn: 0.076

342:	learn: 0.0529383	total: 1.76s	remaining: 3.38s
343:	learn: 0.0527990	total: 1.77s	remaining: 3.38s
344:	learn: 0.0526570	total: 1.78s	remaining: 3.37s
345:	learn: 0.0526214	total: 1.78s	remaining: 3.37s
346:	learn: 0.0524822	total: 1.79s	remaining: 3.36s
347:	learn: 0.0522963	total: 1.79s	remaining: 3.35s
348:	learn: 0.0522270	total: 1.79s	remaining: 3.35s
349:	learn: 0.0520631	total: 1.8s	remaining: 3.34s
350:	learn: 0.0519668	total: 1.8s	remaining: 3.33s
351:	learn: 0.0518833	total: 1.8s	remaining: 3.32s
352:	learn: 0.0518365	total: 1.81s	remaining: 3.31s
353:	learn: 0.0516518	total: 1.81s	remaining: 3.31s
354:	learn: 0.0515759	total: 1.81s	remaining: 3.3s
355:	learn: 0.0514473	total: 1.82s	remaining: 3.29s
356:	learn: 0.0513644	total: 1.82s	remaining: 3.28s
357:	learn: 0.0512648	total: 1.82s	remaining: 3.27s
358:	learn: 0.0511527	total: 1.83s	remaining: 3.26s
359:	learn: 0.0510283	total: 1.83s	remaining: 3.25s
360:	learn: 0.0509884	total: 1.83s	remaining: 3.25s
361:	learn: 0.05

542:	learn: 0.0365935	total: 2.73s	remaining: 2.3s
543:	learn: 0.0365448	total: 2.73s	remaining: 2.29s
544:	learn: 0.0364932	total: 2.74s	remaining: 2.29s
545:	learn: 0.0363879	total: 2.75s	remaining: 2.28s
546:	learn: 0.0363507	total: 2.75s	remaining: 2.28s
547:	learn: 0.0363017	total: 2.75s	remaining: 2.27s
548:	learn: 0.0362419	total: 2.76s	remaining: 2.27s
549:	learn: 0.0361876	total: 2.76s	remaining: 2.26s
550:	learn: 0.0361383	total: 2.77s	remaining: 2.25s
551:	learn: 0.0360778	total: 2.77s	remaining: 2.25s
552:	learn: 0.0360078	total: 2.77s	remaining: 2.24s
553:	learn: 0.0359650	total: 2.78s	remaining: 2.24s
554:	learn: 0.0359142	total: 2.78s	remaining: 2.23s
555:	learn: 0.0358512	total: 2.78s	remaining: 2.22s
556:	learn: 0.0357766	total: 2.79s	remaining: 2.22s
557:	learn: 0.0357550	total: 2.79s	remaining: 2.21s
558:	learn: 0.0356743	total: 2.79s	remaining: 2.2s
559:	learn: 0.0356254	total: 2.8s	remaining: 2.2s
560:	learn: 0.0355693	total: 2.8s	remaining: 2.19s
561:	learn: 0.035

733:	learn: 0.0271536	total: 3.7s	remaining: 1.34s
734:	learn: 0.0271167	total: 3.71s	remaining: 1.34s
735:	learn: 0.0270767	total: 3.71s	remaining: 1.33s
736:	learn: 0.0270639	total: 3.72s	remaining: 1.33s
737:	learn: 0.0270190	total: 3.73s	remaining: 1.32s
738:	learn: 0.0269749	total: 3.74s	remaining: 1.32s
739:	learn: 0.0269204	total: 3.74s	remaining: 1.31s
740:	learn: 0.0268578	total: 3.75s	remaining: 1.31s
741:	learn: 0.0268010	total: 3.75s	remaining: 1.3s
742:	learn: 0.0267381	total: 3.75s	remaining: 1.3s
743:	learn: 0.0267038	total: 3.76s	remaining: 1.29s
744:	learn: 0.0266904	total: 3.77s	remaining: 1.29s
745:	learn: 0.0266570	total: 3.77s	remaining: 1.28s
746:	learn: 0.0266183	total: 3.78s	remaining: 1.28s
747:	learn: 0.0265640	total: 3.78s	remaining: 1.27s
748:	learn: 0.0265291	total: 3.78s	remaining: 1.27s
749:	learn: 0.0264818	total: 3.79s	remaining: 1.26s
750:	learn: 0.0264001	total: 3.8s	remaining: 1.26s
751:	learn: 0.0263850	total: 3.8s	remaining: 1.25s
752:	learn: 0.026

910:	learn: 0.0218323	total: 4.68s	remaining: 457ms
911:	learn: 0.0218214	total: 4.69s	remaining: 452ms
912:	learn: 0.0218141	total: 4.69s	remaining: 447ms
913:	learn: 0.0217844	total: 4.7s	remaining: 442ms
914:	learn: 0.0217466	total: 4.7s	remaining: 437ms
915:	learn: 0.0217263	total: 4.71s	remaining: 432ms
916:	learn: 0.0216881	total: 4.71s	remaining: 427ms
917:	learn: 0.0216496	total: 4.72s	remaining: 421ms
918:	learn: 0.0216029	total: 4.72s	remaining: 416ms
919:	learn: 0.0216016	total: 4.73s	remaining: 411ms
920:	learn: 0.0215598	total: 4.73s	remaining: 406ms
921:	learn: 0.0215386	total: 4.74s	remaining: 401ms
922:	learn: 0.0215277	total: 4.75s	remaining: 396ms
923:	learn: 0.0215206	total: 4.75s	remaining: 391ms
924:	learn: 0.0215100	total: 4.76s	remaining: 386ms
925:	learn: 0.0214973	total: 4.77s	remaining: 381ms
926:	learn: 0.0214519	total: 4.77s	remaining: 376ms
927:	learn: 0.0214254	total: 4.78s	remaining: 371ms
928:	learn: 0.0213967	total: 4.79s	remaining: 366ms
929:	learn: 0.

In [203]:
finalpredd

array([0.06218731, 0.06221815, 0.06997637, ..., 0.13227526, 0.06225586,
       0.08423664])

In [204]:
output_df= pd.DataFrame({
    'bidder_id': final_test_df.bidder_id,
    'Prediction':finalpredd
})
output_df

,bidder_id,Prediction
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,0.062187
1,a921612b85a1494456e74c09393ccb65ylp4y,0.062218
2,6b601e72a4d264dab9ace9d7b229b47479v6i,0.069976
3,eaf0ed0afc9689779417274b4791726cn5udi,0.064805
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,0.066077
...,...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd,0.546117
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,0.075020
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,0.132275
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,0.062256


In [205]:
output_df.to_csv('Facebooksubmission.csv', index= False)